In [5]:
from google.colab import drive
import sys
import os
import time

import pandas as pd

drive.mount('/content/drive')  ### conect to drive
path='/content/drive/MyDrive/cod/RL_DS/Lotka_volterra'
os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/']

In [41]:
from LotkaVolterra import LotkaVolterraModel
from scipy.optimize import minimize ## optimo local
from scipy.optimize import differential_evolution, dual_annealing, basinhopping, shgo, brute ## optimo global


In [17]:
P = [0.002, 0.04, 0.1, 0.0025] # Valor inicial de los parametros
Lim = [[0.0015,0.0025],[0.03,0.05],[0.05,0.15],[0.002,0.003]] #


def objective(P):
  lv=LotkaVolterraModel(*P)
  return -lv.simulate()




-39.97034792759246

In [19]:
result= minimize(objective, P,bounds=Lim, method )

In [24]:
result
objective(result.x)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -80.10988274018814
        x: [ 1.500e-03  5.000e-02  1.500e-01  2.000e-03]
      nit: 1
      jac: [ 6.893e+02 -3.331e+02 -5.753e+02  3.854e+04]
     nfev: 10
     njev: 2
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>

In [26]:
result2= differential_evolution(objective,Lim)

In [28]:
result2.x
objective(result2.x)

-87.88031443429456

In [39]:
# prompt: create a code trying different optimization methods in scipy and creating a table with results
methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'SLSQP', 'trust-constr', 'dogleg', 'trust-ncg', 'trust-exact']
results = []

for method in methods:
    try:
        start_time = time.time()
        result = minimize(objective, P, bounds=Lim, method=method)
        end_time = time.time()
        results.append({
            'Method': method,
            'Success': result.success,
            'Optimal Parameters': result.x,
            'Objective Value': -result.fun, #negate since we minimized the negative
            'Iterations': result.nit,
            'Time (seconds)': end_time - start_time
        })
    except Exception as e:
        print(f"Error with method {method}: {e}")

# Create a DataFrame from the results list

df = pd.DataFrame(results)
df.sort_values(by='Objective Value', ascending=False)


<ipython-input-39-22196bde175c>:8: RuntimeWarning: Method CG cannot handle bounds.
  result = minimize(objective, P, bounds=Lim, method=method)
/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/LotkaVolterra.py:32: RuntimeWarning: overflow encountered in scalar multiply
  wolves_birth = self.alpha * (rabbits * wolves)
/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/LotkaVolterra.py:36: RuntimeWarning: overflow encountered in scalar multiply
  rabbits_death = self.gamma * (rabbits * wolves)
/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/LotkaVolterra.py:29: RuntimeWarning: invalid value encountered in scalar subtract
  wolves += (wolves_birth - wolves_death) * self.dt
/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/LotkaVolterra.py:32: RuntimeWarning: overflow encountered in scalar multiply
  wolves_birth = self.alpha * (rabbits * wolves)
/content/drive/MyDrive/cod/RL_DS/Lotka_volterra/LotkaVolterra.py:36: RuntimeWarning: overflow encountered in scalar multiply
  rabbits_death = self.gam

Error with method dogleg: Jacobian is required for dogleg minimization
Error with method trust-ncg: Jacobian is required for Newton-CG trust-region minimization
Error with method trust-exact: Jacobian is required for trust region exact minimization.


<ipython-input-39-22196bde175c>:8: RuntimeWarning: Method dogleg cannot handle bounds.
  result = minimize(objective, P, bounds=Lim, method=method)
<ipython-input-39-22196bde175c>:8: RuntimeWarning: Method trust-ncg cannot handle bounds.
  result = minimize(objective, P, bounds=Lim, method=method)
<ipython-input-39-22196bde175c>:8: RuntimeWarning: Method trust-exact cannot handle bounds.
  result = minimize(objective, P, bounds=Lim, method=method)


,Method,Success,Optimal Parameters,Objective Value,Iterations,Time (seconds)
1,Powell,True,"[0.0024999957245487895, 0.030000415023117566, ...",85.263357,6,21.172274
5,TNC,True,"[0.0025, 0.042843719085909705, 0.15, 0.002]",84.205548,8,6.864436
0,Nelder-Mead,True,"[0.0024999999999999996, 0.04283313865696524, 0...",84.205514,57,5.905792
4,L-BFGS-B,True,"[0.0015, 0.05, 0.15, 0.002]",80.109883,1,0.438759
6,SLSQP,True,"[0.0016395532784845273, 0.04998436812858472, 0...",61.952507,6,0.439905
2,CG,False,"[0.002, 0.04, 0.1, 0.0025]",39.970348,0,2.848392
3,BFGS,False,"[0.002, 0.04, 0.1, 0.0025]",39.970348,0,2.873263
7,trust-constr,False,"[0.002, 0.04, 0.1, 0.0025]",39.970348,1000,3.356960


In [44]:

# Global optimization methods and their explanations
methods = {
    "differential_evolution": "Stochastic population-based method, robust for complex landscapes.",
    "dual_annealing": "Combines simulated annealing with local search, good for global exploration.",
    "basinhopping": "Combines Metropolis-Hastings with local minimization, explores basins of attraction.",
    "shgo": "Simplicial homology global optimization, explores the topology of the search space.",
    "brute": "Brute-force search (grid-based), guarantees finding the global optimum but can be slow."
}

results = []

for method_name, explanation in methods.items():
    start_time = time.time()

    if method_name == "differential_evolution":
        result = differential_evolution(objective, Lim)
    elif method_name == "dual_annealing":
        result = dual_annealing(objective, bounds=Lim)
    elif method_name == "basinhopping":
        result = basinhopping(objective, P, minimizer_kwargs={"method": "L-BFGS-B", "bounds": Lim})
    elif method_name == "shgo":
        result = shgo(objective, bounds=Lim)
    elif method_name == "brute":
        # Define ranges for brute-force search
        ranges = [slice(start, stop, (stop - start) / 50) for start, stop in Lim]  # Adjust 50 for grid density
        result = brute(objective, ranges=ranges, full_output=True, finish=None)
        result = {  # Create a dictionary like other methods
            'x': result[0],
            'fun': result[1]
        }

    end_time = time.time()

    results.append({
        "Method": method_name,
        "Suitability Explanation": explanation,
        "Time (seconds)": end_time - start_time,
        "Average Wolf Population": -result['fun'] if 'fun' in result else -result.fun,
        "Optimal Parameters": result['x'] if 'x' in result else result.x  # Access 'x' if available
    })

df = pd.DataFrame(results)
print(df)

                   Method                            Suitability Explanation  \
0  differential_evolution  Stochastic population-based method, robust for...   
1          dual_annealing  Combines simulated annealing with local search...   
2            basinhopping  Combines Metropolis-Hastings with local minimi...   
3                    shgo  Simplicial homology global optimization, explo...   

   Time (seconds)  Average Wolf Population           Optimal Parameters  
0       89.220366                87.880314  [0.0025, 0.03, 0.15, 0.002]  
1      403.586849                87.880314  [0.0025, 0.03, 0.15, 0.002]  
2       49.249872                87.880314  [0.0025, 0.03, 0.15, 0.002]  
3        1.183980                87.880314  [0.0025, 0.03, 0.15, 0.002]  
